# Process Data
- get_batch

In [1]:
from process_data import *
from params import *

In [2]:
params = Params()
params.DATA_PATH = '../data/target/compar/'
graph_name = 'bigcomp'
batch_gen = ProcessData(params)

bigcomp
load data : ../data/target/compar/train.pkl
5206 5206
load data : ../data/target/compar/dev.pkl
729 729
load data : ../data/target/compar/test.pkl
1445 1445
[completed] load data, feature_dim:  6373


In [12]:
np.shape(batch_gen.train[0][0])[0]

6373

In [5]:
list_d, list_l = batch_gen.get_batch(
                            data=batch_gen.test,
                            batch_size=10,
                            is_test=False,
                            start_index = 0
                            )
print(np.shape(list_d), np.shape(list_l))

(10, 6373) (10, 6)


In [9]:
np.shape(list_d)

(10, 6373)

# model

In [1]:
import os
from model_cnn import *
from process_data import *
from params import *
# from train import *

params = Params()
params.DATA_PATH = '../data/target/compar/'
params.FEATURE_DIM = 6373
graph_name = 'bigcomp'
##########################################    
params.IS_RESULT_LOGGIN = False
##########################################          
batch_gen = ProcessData(params)

params.BATCH_SIZE = 2
params.LR = 1e-3

model = SimpleCNN(params=params)

model.build_graph()
# model._create_placeholders()
# model._apply_CNN()
# model._create_output_layers()

bigcomp
load data : ../data/target/compar/train.pkl
5206 5206
load data : ../data/target/compar/dev.pkl
729 729
load data : ../data/target/compar/test.pkl
1445 1445
[completed] load data, feature_dim:  6373
[launch] placeholders
[INFO] CNN
[launch] create output projection layer
[launch] create optimizer
[launch] create summary


In [2]:
tf.trainable_variables()

[<tf.Variable 'cnn0/kernel:0' shape=(100, 1, 32) dtype=float32_ref>,
 <tf.Variable 'cnn0/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'cnn1/kernel:0' shape=(200, 1, 32) dtype=float32_ref>,
 <tf.Variable 'cnn1/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'cnn2/kernel:0' shape=(300, 1, 32) dtype=float32_ref>,
 <tf.Variable 'cnn2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'cnn3/kernel:0' shape=(400, 1, 32) dtype=float32_ref>,
 <tf.Variable 'cnn3/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(128, 32) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(32, 6) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(6,) dtype=float32_ref>]

In [3]:
saver  = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

# if exists check point, starts from the check point
ckpt = tf.train.get_checkpoint_state(os.path.dirname('./save/'))
if ckpt and ckpt.model_checkpoint_path:
    print ('from check point!!!')
    saver.restore(sess, ckpt.model_checkpoint_path)

In [4]:
list_d, list_l = batch_gen.get_batch(
                                    data=batch_gen.dev,
                                    batch_size=model.params.BATCH_SIZE,
                                    is_test=True,
                                    start_index = 0
                                    )

# prepare data which will be push from pc to placeholder
input_feed = {}

input_feed[model.batch_d]     = list_d
input_feed[model.batch_l]     = list_l
input_feed[model.phase]     = False
input_feed[model.dr_prob] = 1.0

In [9]:
r1, r2, r3 = sess.run( [
                model.output,
                model.batch_loss,
                model.loss
               ], input_feed)

print(np.shape(r1), np.shape(r2), np.shape(r3))
# print(list_len_q)
# print(list_len_a)

(2, 6) (2,) ()


In [10]:
train_model(params, model, batch_gen, num_train_steps=2, valid_freq=1)

NameError: name 'train_model' is not defined

# Evaluation

In [1]:
import os
from model import *
from process_data import *
from params import *
from evaluation import *
# from train import *

params = Params()
params.data_path = '../data/target/fold_1/'
graph_name = 'bigcomp'
##########################################    
params.IS_RESULT_LOGGIN = False
##########################################          
batch_gen = ProcessData(params)

params.batch_size = 2
params.lr = 1e-3

model = SimpleMLP(params=params)

model.build_graph()

bigcomp
load data : ../data/target/fold_1/train.pkl
5206 5206
load data : ../data/target/fold_1/dev.pkl
729 729
load data : ../data/target/fold_1/test.pkl
1445 1445
[completed] load data
[launch] placeholders
[INFO] MLP
[launch] MLP (reuse, scope):  False L1
[launch] create output projection layer
[launch] create optimizer
[launch] create summary


In [2]:
saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

# if exists check point, starts from the check point
ckpt = tf.train.get_checkpoint_state(os.path.dirname('./save/best-model/'))
if ckpt and ckpt.model_checkpoint_path:
    print ('from check point!!!')
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("no pre-model available")

no pre-model available


In [3]:
data         = batch_gen.dev
test_loss, test_MAP, test_MRR, _ = run_test(sess=sess,
                                     model=model,
                                     batch_gen=batch_gen,
                                     data=data
                                           )
print(test_MAP, test_MRR)

0.12488455453227748 0.1083170493404983


/home/dato/anaconda2/envs/tf14_p35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [4]:
list_d, list_l = batch_gen.get_batch(
                                    data=batch_gen.dev,
                                    batch_size=2,
                                    is_test=True,
                                    start_index = 0
                                    )

# prepare data which will be push from pc to placeholder
input_feed = {}

input_feed[model.batch_d] = list_d
input_feed[model.batch_l] = list_l
input_feed[model.dr_prob] = 1.0

In [6]:
r1, r2, r3 = sess.run( [
                model.batch_preds,
                model.y_labels,
                model.loss
               ], input_feed)

print(np.shape(r1), np.shape(r2), np.shape(r3))
# print(list_len_q)
# print(list_len_a)

(2, 6) (2, 6) ()


In [7]:
r1[0]

array([0.49230745, 0.49239844, 0.4924431 , 0.5004191 , 0.5028848 ,
       0.5003315 ], dtype=float32)

In [10]:
np.argmax(r2, axis=1)

array([4, 5])

In [11]:
data_reshape = tf.reshape( model.batch_d, [model.params.BATCH_SIZE, -1, 1])
cnn_filters=np.ndarray([30,1,1], dtype=np.float32)

cnn_1d = tf.layers.conv1d(
    inputs=data_reshape,
    filters=64,
    kernel_size=300,
    strides=10,
    padding="SAME",
    activation=tf.nn.relu
)
print(cnn_1d)

Tensor("conv1d_2/Relu:0", shape=(2, 638, 64), dtype=float32)


In [16]:
pool = tf.layers.max_pooling1d(
    inputs=cnn_1d,
    pool_size=638,
    strides=638,
    padding='SAME'
)
print(pool)

Tensor("max_pooling1d_5/Squeeze:0", shape=(2, 1, 64), dtype=float32)


In [17]:
pool_avg = tf.layers.average_pooling1d(
    inputs=cnn_1d,
    pool_size=638,
    strides=638,
    padding='SAME'
)
print(pool_avg)

Tensor("average_pooling1d_1/Squeeze:0", shape=(2, 1, 64), dtype=float32)
